# Machine Learning with PySpark

Spark is a powerful, general purpose tool for working with Big Data. Spark transparently handles the distribution of compute tasks across a cluster. This means that operations are fast, but it also allows you to focus on the analysis rather than worry about technical details. In this course you'll learn how to get data into Spark and then delve into the three fundamental Spark Machine Learning algorithms: Linear Regression, Logistic Regression/Classifiers, and creating pipelines. Along the way you'll analyse a large dataset of flight delays and spam text messages. With this background you'll be ready to harness the power of Spark and apply it on your own Machine Learning projects!

## Table of Contents

- [Introduction & Data Loading](#intro)
- [Classification](#class)
- [Regression](#reg)
- [Ensembles & Pipelines](#ens)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

path = "data/dc35/"

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")
print(sc)

<SparkContext master=local appName=First App>


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('First App').getOrCreate()

In [5]:
# Return spark version
print(spark.version)

# Return python version
import sys
print(sys.version_info)

2.4.4
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


---
<a id='intro'></a>

## Machine Learning & Spark

<img src="images/spark5_001.png" alt="" style="width: 800px;"/>

<img src="images/spark5_002.png" alt="" style="width: 800px;"/>

<img src="images/spark5_003.png" alt="" style="width: 800px;"/>

<img src="images/spark5_004.png" alt="" style="width: 800px;"/>

<img src="images/spark5_005.png" alt="" style="width: 800px;"/>

## Connecting to Spark

<img src="images/spark5_006.png" alt="" style="width: 800px;"/>

<img src="images/spark5_007.png" alt="" style="width: 800px;"/>

<img src="images/spark5_008.png" alt="" style="width: 800px;"/>

<img src="images/spark5_009.png" alt="" style="width: 800px;"/>

<img src="images/spark5_010.png" alt="" style="width: 800px;"/>

## Creating a SparkSession

In this exercise, you'll spin up a local Spark cluster using all available cores. The cluster will be accessible via a `SparkSession` object.

The SparkSession class has a `builder` attribute, which is an instance of the Builder class. The Builder class exposes three important methods that let you:

- specify the location of the master node;
- name the application (optional); and
- retrieve an existing SparkSession or, if there is none, create a new one.

The SparkSession class has a `version` attribute which gives the version of Spark.

Find out more about SparkSession [here](https://spark.apache.org/docs/2.3.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession).

Once you are finished with the cluster, it's a good idea to shut it down, which will free up its resources, making them available for other processes.

- Import the SparkSession class from pyspark.sql.
- Create a SparkSession object connected to a local cluster. Use all available cores. Name the application 'test'.
- Use the SparkSession object to retrieve the version of Spark running on the cluster.
- Shut down the cluster.

In [2]:
# Import the PySpark module
from pyspark.sql import SparkSession

# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('test') \
                    .getOrCreate()

# What version of Spark?
print(spark.version)

# Terminate the cluster
spark.stop()

2.4.4


## Loading Data

<img src="images/spark5_011.png" alt="" style="width: 800px;"/>

<img src="images/spark5_012.png" alt="" style="width: 800px;"/>

<img src="images/spark5_013.png" alt="" style="width: 800px;"/>

<img src="images/spark5_014.png" alt="" style="width: 800px;"/>

<img src="images/spark5_015.png" alt="" style="width: 800px;"/>

<img src="images/spark5_016.png" alt="" style="width: 800px;"/>

<img src="images/spark5_017.png" alt="" style="width: 800px;"/>

<img src="images/spark5_018.png" alt="" style="width: 800px;"/>

<img src="images/spark5_019.png" alt="" style="width: 800px;"/>

## Loading flights data

In this exercise you're going to load some airline flight data from a CSV file. To ensure that the exercise runs quickly these data have been trimmed down to only 50 000 records. You can get a larger dataset in the same format [here]().

Notes on CSV format:

- fields are separated by a comma (this is the default separator) and
- missing data are denoted by the string 'NA'.

Data dictionary:

- mon — month (integer between 1 and 12)
- dom — day of month (integer between 1 and 31)
- dow — day of week (integer; 1 = Monday and 7 = Sunday)
- org — origin airport (IATA code)
- mile — distance (miles)
- carrier — carrier (IATA code)
- depart — departure time (decimal hour)
- duration — expected duration (minutes)
- delay — delay (minutes)

pyspark has been imported for you and the session has been initialized.

- Read data from a CSV file called 'flights.csv'. Assign data types to columns automatically. Deal with missing data.
- How many records are in the data?
- Take a look at the first five records.
- What data types have been assigned to the columns? Do these look correct?

In [7]:
# Read data from CSV file
flights = spark.read.csv(path+'flights-larger.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % flights.count())

# View the first five records
flights.show(5)

# Check column data types
flights.dtypes

The data contain 275000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| null|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



[('mon', 'int'),
 ('dom', 'int'),
 ('dow', 'int'),
 ('carrier', 'string'),
 ('flight', 'int'),
 ('org', 'string'),
 ('mile', 'int'),
 ('depart', 'double'),
 ('duration', 'int'),
 ('delay', 'int')]

## Loading SMS spam data

You've seen that it's possible to infer data types directly from the data. Sometimes it's convenient to have direct control over the column types. You do this by defining an explicit schema.

The file sms.csv contains a selection of SMS messages which have been classified as either 'spam' or 'ham'. These data have been adapted from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). There are a total of 5574 SMS, of which 747 have been labelled as spam.

Notes on CSV format:

- no header record and
- fields are separated by a semicolon (this is not the default separator).

Data dictionary:

- id — record identifier
- text — content of SMS message
- label — spam or ham (integer; 0 = ham and 1 = spam)

Instructions

- Specify the data schema, giving columns names ("id", "text", and "label") and column types.
- Read data from a delimited file called "sms.csv".
- Print the schema for the resulting DataFrame.

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Specify column names and types
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])

# Load data from a delimited file
sms = spark.read.csv(path+'sms.csv', sep=';', header=False, schema=schema)

# Print schema of DataFrame
sms.printSchema()

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



---
<a id='class'></a>

## Classification

## Data Preparation

<img src="images/spark5_020.png" alt="" style="width: 800px;"/>

<img src="images/spark5_021.png" alt="" style="width: 800px;"/>

<img src="images/spark5_022.png" alt="" style="width: 800px;"/>

<img src="images/spark5_023.png" alt="" style="width: 800px;"/>

<img src="images/spark5_024.png" alt="" style="width: 800px;"/>

<img src="images/spark5_025.png" alt="" style="width: 800px;"/>

<img src="images/spark5_026.png" alt="" style="width: 800px;"/>

## Removing columns and rows

You previously loaded airline flight data from a CSV file. You're going to develop a model which will predict whether or not a given flight will be delayed.

In this exercise you need to trim those data down by:

- removing an uninformative column and
- removing rows which do not have information about whether or not a flight was delayed.

Instructions

- Remove the flight column.
- Find out how many records have missing values in the delay column.
- Remove records with missing values in the delay column.
- Remove records with missing values in any column and get the number of remaining rows.

In [10]:
# Remove the 'flight' column
flights = flights.drop('flight')

# Number of records with missing 'delay' values
flights.filter('delay IS NULL').count()

# Remove records with missing 'delay' values
flights = flights.filter('delay IS NOT NULL')

# Remove records with missing values in any column and get the number of remaining rows
flights = flights.dropna()
print(flights.count())

258289


## Column manipulation

The Federal Aviation Administration (FAA) considers a flight to be "delayed" when it arrives 15 minutes or more after its scheduled time.

The next step of preparing the flight data has two parts:

1. convert the units of distance, replacing the mile column with a km column; and
1. create a Boolean column indicating whether or not a flight was delayed.

Instructions

- Import a function which will allow you to round a number to a specific number of decimal places.
- Derive a new km column from the mile column, rounding to zero decimal places. One mile is 1,60934 km.
- Remove the mile column.
- Create a label column with a value of 1 indicating the delay was 15 minutes or more and 0 otherwise.

In [11]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights_km = flights.withColumn('km', round(flights.mile * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check first five records
flights_km.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|
|  2| 14|  5|     B6|JFK| 21.17|     365|   60|3618.0|    1|
|  5| 25|  3|     WN|SJC| 12.92|      85|   22| 621.0|    1|
|  3| 28|  1|     B6|LGA| 13.33|     182|   70|1732.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



## Categorical columns

In the flights data there are two columns, carrier and org, which hold categorical data. You need to transform those columns into indexed numerical values.

- Import the appropriate class and create an indexer object to transform the carrier column from a string to an numeric index.
- Prepare the indexer object on the flight data.
- Use the prepared indexer to create the numeric index column.
- Repeat the process for the org column.

In [17]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights_km)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights_km)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

In [18]:
flights_indexed.show(10)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|        2.0|    0.0|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|        2.0|    0.0|
|  2| 14|  5|     B6|JFK| 21.17|     365|   60|3618.0|    1|        4.0|    2.0|
|  5| 25|  3|     WN|SJC| 12.92|      85|   22| 621.0|    1|        3.0|    5.0|
|  3| 28|  1|     B6|LGA| 13.33|     182|   70|1732.0|    1|        4.0|    3.0|
|  5| 28|  6|     B6|ORD|  9.58|     130|   47|1191.0|    1|        4.0|    0.0|
|  1| 19|  2|     UA|SFO| 12.75|     123|  135|1093.0|    1|        0.0|    1.0|
|  8|  5|  5|     US|LGA|  13.0|      71|  -10| 344.0|    0|        6.0|    3.0|
|  5| 27|  5|     AA|ORD| 14.42|     195|  -11|1926.0|    0|        1.0|    0.0|
|  8| 20|  6|     B6|JFK| 14

## Assembling columns

The final stage of data preparation is to consolidate all of the predictor columns into a single column.

At present our data has the following predictor columns:

- mon, dom and dow
- carrier_idx (derived from carrier)
- org_idx (derived from org)
- km
- depart
- duration

Instructions

- Import the class which will assemble the predictors.
- Create an assembler object that will allow you to merge the predictors columns into a single column.
- Use the assembler to generate a new consolidated column.

In [19]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=['mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration'], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_indexed)

# Check the resulting column
flights_assembled.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[10.0,10.0,1.0,2.0,0.0,253.0,8.18,51.0]  |27   |
|[11.0,22.0,1.0,2.0,0.0,1188.0,7.17,127.0]|-19  |
|[2.0,14.0,5.0,4.0,2.0,3618.0,21.17,365.0]|60   |
|[5.0,25.0,3.0,3.0,5.0,621.0,12.92,85.0]  |22   |
|[3.0,28.0,1.0,4.0,3.0,1732.0,13.33,182.0]|70   |
+-----------------------------------------+-----+
only showing top 5 rows



## Decision Tree

<img src="images/spark5_027.png" alt="" style="width: 800px;"/>

<img src="images/spark5_028.png" alt="" style="width: 800px;"/>

<img src="images/spark5_029.png" alt="" style="width: 800px;"/>

<img src="images/spark5_030.png" alt="" style="width: 800px;"/>

<img src="images/spark5_031.png" alt="" style="width: 800px;"/>

## Train/test split

To objectively assess a Machine Learning model you need to be able to test it on an independent set of data. You can't use the same data that you used to train the model: of course the model will perform (relatively) well on those data!

You will split the data into two components:

- training data (used to train the model) and
- testing data (used to test the model).

Instructions

- Randomly split the flights data into two sets with 80:20 proportions. For repeatability set a random number seed of 17 for the split.
- Check that the testing data has roughly 80% of the records from the original data.

In [20]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights_assembled.count()
print(training_ratio)

0.7993991226881516


## Build a Decision Tree

Now that you've split the flights data into training and testing sets, you can use the training set to fit a Decision Tree model.

The data are available as flights_train and flights_test.

- Import the class for creating a Decision Tree classifier.
- Create a classifier object and fit it to the training data.
- Make predictions for the testing data and take a look at the predictions.

In [21]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|0    |1.0       |[0.49063291139240506,0.5093670886075949]|
|1    |1.0       |[0.49063291139240506,0.5093670886075949]|
|1    |1.0       |[0.3706408408031927,0.6293591591968073] |
|0    |1.0       |[0.3706408408031927,0.6293591591968073] |
|1    |1.0       |[0.3706408408031927,0.6293591591968073] |
+-----+----------+----------------------------------------+
only showing top 5 rows



## Evaluate the Decision Tree

You can assess the quality of your model by evaluating how well it performs on the testing data. Because the model was not trained on these data, this represents an objective assessment of the model.

A `confusion matrix` gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:

- **True Negatives (TN)** — model predicts negative outcome & known outcome is negative
- **True Positives (TP)** — model predicts positive outcome & known outcome is positive
- **False Negatives (FN)** — model predicts negative outcome but known outcome is positive
- **False Positives (FP)** — model predicts positive outcome but known outcome is negative.

Instructions

- Create a confusion matrix by counting the combinations of label and prediction. Display the result.
- Count the number of True Negatives, True Positives, False Negatives and False Positives.
- Calculate the accuracy.

In [22]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TP+TN)/(TP+TN+FN+FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 6968|
|    0|       0.0|13752|
|    1|       1.0|19197|
|    0|       1.0|11896|
+-----+----------+-----+

0.6359214868855306


The accuracy is decent but there are a lot of false predictions. We can make this model better!

**Precision** is a proportion of predictions which are correct: TP / (TP + FP)
**Recall** is a proportion of positive targets which are correctly predicted: TP / (TP + FN)

## Logistic Regression

<img src="images/spark5_032.png" alt="" style="width: 800px;"/>

<img src="images/spark5_033.png" alt="" style="width: 800px;"/>

<img src="images/spark5_034.png" alt="" style="width: 800px;"/>

<img src="images/spark5_035.png" alt="" style="width: 800px;"/>

<img src="images/spark5_036.png" alt="" style="width: 800px;"/>

<img src="images/spark5_037.png" alt="" style="width: 800px;"/>

<img src="images/spark5_038.png" alt="" style="width: 800px;"/>

<img src="images/spark5_039.png" alt="" style="width: 800px;"/>

<img src="images/spark5_040.png" alt="" style="width: 800px;"/>

<img src="images/spark5_041.png" alt="" style="width: 800px;"/>

<img src="images/spark5_042.png" alt="" style="width: 800px;"/>

## Build a Logistic Regression model

You've already built a Decision Tree model using the flights data. Now you're going to create a Logistic Regression model on the same data.

The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

Although you have a variety of predictors at your disposal, you'll only use the mon, depart and duration columns for the moment. These are numerical features which can immediately be used for a Logistic Regression model. You'll need to do a little more work before you can include categorical features. Stay tuned!

The data have been split into training and testing sets and are available as flights_train and flights_test.

- Import the class for creating a Logistic Regression classifier.
- Create a classifier object and train it on the training data.
- Make predictions for the testing data and create a confusion matrix.

In [25]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 9316|
|    0|       0.0|14870|
|    1|       1.0|16849|
|    0|       1.0|10778|
+-----+----------+-----+



Now let's unpack that confusion matrix.

## Evaluate the Logistic Regression model

`Accuracy` is generally not a very reliable metric because it can be biased by the most common target class.

There are two other useful metrics:

- precision and
- recall.

Check the slides for this lesson to get the relevant expressions.

`Precision` is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?

`Recall` is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?

The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.

The components of the confusion matrix are available as TN, TP, FN and FP, as well as the object prediction.

- Find the precision and recall.
- Create a multi-class evaluator and evaluate weighted precision.
- Create a binary evaluator and evaluate AUC using the "areaUnderROC" metric.

In [26]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 0.61
recall    = 0.64


In [28]:
weighted_precision

0.6123217775889148

In [29]:
auc

0.6521151892013184

## Turning Text into Tables

<img src="images/spark5_043.png" alt="" style="width: 800px;"/>

<img src="images/spark5_044.png" alt="" style="width: 800px;"/>

<img src="images/spark5_045.png" alt="" style="width: 800px;"/>

<img src="images/spark5_046.png" alt="" style="width: 800px;"/>

<img src="images/spark5_047.png" alt="" style="width: 800px;"/>

<img src="images/spark5_048.png" alt="" style="width: 800px;"/>

<img src="images/spark5_049.png" alt="" style="width: 800px;"/>

<img src="images/spark5_050.png" alt="" style="width: 800px;"/>

<img src="images/spark5_051.png" alt="" style="width: 800px;"/>

## Punctuation, numbers and tokens

At the end of the previous chapter you loaded a dataset of SMS messages which had been labeled as either "spam" (label 1) or "ham" (label 0). You're now going to use those data to build a classifier model.

But first you'll need to prepare the SMS messages as follows:

- remove punctuation and numbers
- tokenize (split into individual words)
- remove stop words
- apply the hashing trick
- convert to TF-IDF representation.

In this exercise you'll remove punctuation and numbers, then tokenize the messages.

The SMS data are available as sms.

- Import the function to replace regular expressions and the feature to tokenize.
- Replace all punctuation characters from the text column with a space. Do the same for all numbers in the text column.
- Split the text column into tokens. Name the output column words.

In [32]:
sms.show()

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|Sorry, I'll call ...|    0|
|  2|Dont worry. I gue...|    0|
|  3|Call FREEPHONE 08...|    1|
|  4|Win a 1000 cash p...|    1|
|  5|Go until jurong p...|    0|
|  6|Ok lar... Joking ...|    0|
|  7|Free entry in 2 a...|    1|
|  8|U dun say so earl...|    0|
|  9|Nah I don't think...|    0|
| 10|FreeMsg Hey there...|    1|
| 11|Even my brother i...|    0|
| 12|As per your reque...|    0|
| 13|WINNER!! As a val...|    1|
| 14|Had your mobile 1...|    1|
| 15|I'm gonna be home...|    0|
| 16|SIX chances to wi...|    1|
| 17|URGENT! You have ...|    1|
| 18|I've been searchi...|    0|
| 19|I HAVE A DATE ON ...|    0|
| 20|XXXMobileMovieClu...|    1|
+---+--------------------+-----+
only showing top 20 rows



In [33]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=False)

+---+----------------------------------+-----+------------------------------------------+
|id |text                              |label|words                                     |
+---+----------------------------------+-----+------------------------------------------+
|1  |Sorry I'll call later in meeting  |0    |[sorry, i'll, call, later, in, meeting]   |
|2  |Dont worry I guess he's busy      |0    |[dont, worry, i, guess, he's, busy]       |
|3  |Call FREEPHONE now                |1    |[call, freephone, now]                    |
|4  |Win a cash prize or a prize worth |1    |[win, a, cash, prize, or, a, prize, worth]|
+---+----------------------------------+-----+------------------------------------------+
only showing top 4 rows



## Stop words and hashing

The next steps will be to remove stop words and then apply the hashing trick, converting the results into a TF-IDF.

A quick reminder about these concepts:

- `The hashing trick` provides a fast and space-efficient way to map a very large (possibly infinite) set of items (in this case, all words contained in the SMS messages) onto a smaller, finite number of values.
- `The TF-IDF matrix` reflects how important a word is to each document. It takes into account both the frequency of the word within each document but also the frequency of the word across all of the documents in the collection.

The tokenized SMS data are stored in sms in a column named words. You've cleaned up the handling of spaces in the data so that the tokenized text is neater.

- Import the StopWordsRemover, HashingTF and IDF classes.
- Create a StopWordsRemover object (input column words, output column terms). Apply to sms.
- Create a HashingTF object (input results from previous step, output column hash). Apply to wrangled.
- Create an IDF object (input results from previous step, output column features). Apply to wrangled.

In [35]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms')\
      .transform(wrangled)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024)\
      .transform(wrangled)

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features')\
      .fit(wrangled).transform(wrangled)
      
tf_idf.select('terms', 'features').show(4, truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------+
|terms                           |features                                                                                            |
+--------------------------------+----------------------------------------------------------------------------------------------------+
|[sorry, call, later, meeting]   |(1024,[138,344,378,1006],[2.2391682769656747,2.892706319430574,3.684405173719015,4.244020961654438])|
|[dont, worry, guess, busy]      |(1024,[53,233,329,858],[4.618714411095849,3.557143394108088,4.618714411095849,4.937168142214383])   |
|[call, freephone]               |(1024,[138,396],[2.2391682769656747,3.3843005812686773])                                            |
|[win, cash, prize, prize, worth]|(1024,[31,69,387,428],[3.7897656893768414,7.284881949239966,4.4671645129686475,3.898659777615979])  |
+--------------------------------+--------------

## Training a spam classifier

The SMS data have now been prepared for building a classifier. Specifically, this is what you have done:

- removed numbers and punctuation
- split the messages into words (or "tokens")
- removed stop words
- applied the hashing trick and
- converted to a TF-IDF representation.

Next you'll need to split the TF-IDF data into training and testing sets. Then you'll use the training data to fit a Logistic Regression model and finally evaluate the performance of that model on the testing data.

The data are stored in tf_idf and LogisticRegression has been imported for you.

- Split the data into training and testing sets in a 4:1 ratio. Set the random number seed to 13 to ensure repeatability.
- Create a LogisticRegression object and fit it to the training data.
- Generate predictions on the testing data.
- Use the predictions to form a confusion matrix.

In [36]:
tf_idf.show(2)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
| id|                text|label|               words|               terms|                hash|            features|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|  1|Sorry I'll call l...|    0|[sorry, i'll, cal...|[sorry, call, lat...|(1024,[138,344,37...|(1024,[138,344,37...|
|  2|Dont worry I gues...|    0|[dont, worry, i, ...|[dont, worry, gue...|(1024,[53,233,329...|(1024,[53,233,329...|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [37]:
# Split the data into training and testing sets
sms_train, sms_test = tf_idf.randomSplit([0.8, 0.2], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   47|
|    0|       0.0|  987|
|    1|       1.0|  124|
|    0|       1.0|    3|
+-----+----------+-----+



---
<a id='reg'></a>

## Regression

## One-Hot Encoding

<img src="images/spark5_052.png" alt="" style="width: 800px;"/>

<img src="images/spark5_053.png" alt="" style="width: 800px;"/>

<img src="images/spark5_054.png" alt="" style="width: 800px;"/>

<img src="images/spark5_055.png" alt="" style="width: 800px;"/>

<img src="images/spark5_056.png" alt="" style="width: 800px;"/>

<img src="images/spark5_057.png" alt="" style="width: 800px;"/>

<img src="images/spark5_058.png" alt="" style="width: 800px;"/>

<img src="images/spark5_059.png" alt="" style="width: 800px;"/>

## Encoding flight origin

The `org` column in the flights data is a categorical variable giving the airport from which a flight departs.

- ORD — O'Hare International Airport (Chicago)
- SFO — San Francisco International Airport
- JFK — John F Kennedy International Airport (New York)
- LGA — La Guardia Airport (New York)
- SMF — Sacramento
- SJC — San Jose
- TUS — Tucson International Airport
- OGG — Kahului (Hawaii)

Obviously this is only a small subset of airports. Nevertheless, since this is a categorical variable, it needs to be one-hot encoded before it can be used in a regression model.

The data are in a variable called flights. You have already used a string indexer to create a column of indexed values corresponding to the strings in org.

- Import the one-hot encoder class.
- Create an one-hot encoder instance, naming the output column 'org_dummy'.
- Apply the one-hot encoder to the flights data.
- Generate a summary of the mapping from categorical values to binary encoded dummy variables. Include only unique values and order by org_idx.

In [38]:
flights.show(2)

+---+---+---+-------+---+----+------+--------+-----+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|
+---+---+---+-------+---+----+------+--------+-----+
| 10| 10|  1|     OO|ORD| 157|  8.18|      51|   27|
| 11| 22|  1|     OO|ORD| 738|  7.17|     127|  -19|
+---+---+---+-------+---+----+------+--------+-----+
only showing top 2 rows



In [39]:
flights_indexed.show(2)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|        2.0|    0.0|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|        2.0|    0.0|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
only showing top 2 rows



In [40]:
# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoderEstimator

# Create an instance of the one hot encoder
onehot = OneHotEncoderEstimator(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights_indexed)
flights_onehot = onehot.transform(flights_indexed)

# Check the results
flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SMF|    4.0|(7,[4],[1.0])|
|SJC|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



Note that one of the category levels, OGG, does not get a dummy variable.

## Regression

<img src="images/spark5_060.png" alt="" style="width: 800px;"/>

<img src="images/spark5_061.png" alt="" style="width: 800px;"/>

<img src="images/spark5_062.png" alt="" style="width: 800px;"/>

<img src="images/spark5_063.png" alt="" style="width: 800px;"/>

<img src="images/spark5_064.png" alt="" style="width: 800px;"/>

<img src="images/spark5_065.png" alt="" style="width: 800px;"/>

<img src="images/spark5_066.png" alt="" style="width: 800px;"/>

<img src="images/spark5_067.png" alt="" style="width: 800px;"/>

<img src="images/spark5_068.png" alt="" style="width: 800px;"/>

<img src="images/spark5_069.png" alt="" style="width: 800px;"/>

<img src="images/spark5_070.png" alt="" style="width: 800px;"/>

<img src="images/spark5_071.png" alt="" style="width: 800px;"/>

<img src="images/spark5_072.png" alt="" style="width: 800px;"/>

<img src="images/spark5_073.png" alt="" style="width: 800px;"/>

## Flight duration model: Just distance

In this exercise you'll `build a regression model to predict flight duration` (the duration column).

For the moment you'll keep the model simple, including only the distance of the flight (the km column) as a predictor.

The data are in flights_small. The first few records are displayed in the terminal. These data have also been split into training and testing sets and are available as flights_train and flights_test.

- Create a linear regression object. Specify the name of the label column. Fit it to the training data.
- Make predictions on the testing data.
- Create a regression evaluator object and use it to evaluate RMSE on the testing data.

```
Subset from the flights DataFrame:

+------+--------+--------+
|km    |features|duration|
+------+--------+--------+
|3465.0|[3465.0]|351     |
|509.0 |[509.0] |82      |
|542.0 |[542.0] |82      |
|1989.0|[1989.0]|195     |
|415.0 |[415.0] |65      |
+------+--------+--------+
only showing top 5 rows
```

In [44]:
flights_small = flights_indexed.select('km', 'duration')
flights_small.show(2)

+------+--------+
|    km|duration|
+------+--------+
| 253.0|      51|
|1188.0|     127|
+------+--------+
only showing top 2 rows



In [45]:
flights_small = VectorAssembler(inputCols=['km'], outputCol='features').transform(flights_small)
flights_small.show(2)

+------+--------+--------+
|    km|duration|features|
+------+--------+--------+
| 253.0|      51| [253.0]|
|1188.0|     127|[1188.0]|
+------+--------+--------+
only showing top 2 rows



In [46]:
flights_small = flights_small.select('km', 'features', 'duration')
flights_small.show(2)

+------+--------+--------+
|    km|features|duration|
+------+--------+--------+
| 253.0| [253.0]|      51|
|1188.0|[1188.0]|     127|
+------+--------+--------+
only showing top 2 rows



In [47]:
flights_train_small, flights_test_small = flights_small.randomSplit([0.8, 0.2], seed=17)

In [48]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train_small)

# Create predictions for the testing data and take a look at the predictions
predictions = regression.transform(flights_test_small)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration').evaluate(predictions)

+--------+-----------------+
|duration|prediction       |
+--------+-----------------+
|43      |52.24222303359208|
|43      |52.24222303359208|
|43      |52.24222303359208|
|43      |52.24222303359208|
|44      |52.24222303359208|
+--------+-----------------+
only showing top 5 rows



17.034699083996244

## Interpreting the coefficients

The linear regression model for flight duration as a function of distance takes the form
```
duration=α+β×distance
```
where

- α — intercept (component of duration which does not depend on distance) and
- β — coefficient (rate at which duration increases as a function of distance; also called the slope).

By looking at the coefficients of your model you will be able to infer

- how much of the average flight duration is actually spent on the ground and
- what the average speed is during a flight.

The linear regression model is available as regression.

- What's the intercept?
- What are the coefficients? This is a vector.
- Extract the slope for distance by indexing into the vector.
- Find the average speed in km per hour.

In [49]:
# Intercept (average minutes on ground)
inter = regression.intercept
print(inter)

# Coefficients
coefs = regression.coefficients
print(coefs)

# Average minutes per km
minutes_per_km = regression.coefficients[0]
print(minutes_per_km)

# Average speed in km per hour
avg_speed = 60 / minutes_per_km
print(avg_speed)

44.064587755035575
[0.07571884517181952]
0.07571884517181952
792.4051121467758


The average speed of a commercial jet is around 850 km/hour. But you got that already from the data!

## Flight duration model: Adding origin airport

Some airports are busier than others. Some airports are bigger than others too. Flights departing from large or busy airports are likely to spend more time taxiing or waiting for their takeoff slot. So it stands to reason that the duration of a flight might depend not only on the distance being covered but also the airport from which the flight departs.

You are going to make the regression model a little more sophisticated by including the departure airport as a predictor.

These data have been split into training and testing sets and are available as flights_train and flights_test. The origin airport, stored in the org column, has been indexed into org_idx, which in turn has been one-hot encoded into org_dummy. The first few records are displayed in the terminal.

- Fit a linear regression model to the training data.
- Make predictions for the testing data.
- Calculate the RMSE for predictions on the testing data.

```
Subset from the flights DataFrame:

+------+-------+-------------+----------------------+
|km    |org_idx|org_dummy    |features              |
+------+-------+-------------+----------------------+
|3465.0|2.0    |(7,[2],[1.0])|(8,[0,3],[3465.0,1.0])|
|509.0 |0.0    |(7,[0],[1.0])|(8,[0,1],[509.0,1.0]) |
|542.0 |1.0    |(7,[1],[1.0])|(8,[0,2],[542.0,1.0]) |
|1989.0|0.0    |(7,[0],[1.0])|(8,[0,1],[1989.0,1.0])|
|415.0 |0.0    |(7,[0],[1.0])|(8,[0,1],[415.0,1.0]) |
+------+-------+-------------+----------------------+
only showing top 5 rows
```

In [51]:
flights_onehot.show(2)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|    org_dummy|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|        2.0|    0.0|(7,[0],[1.0])|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|        2.0|    0.0|(7,[0],[1.0])|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+
only showing top 2 rows



In [58]:
flights_small2 = flights_onehot.select('km', 'org_idx', 'org_dummy', 'duration')
flights_small2.show(2)

+------+-------+-------------+--------+
|    km|org_idx|    org_dummy|duration|
+------+-------+-------------+--------+
| 253.0|    0.0|(7,[0],[1.0])|      51|
|1188.0|    0.0|(7,[0],[1.0])|     127|
+------+-------+-------------+--------+
only showing top 2 rows



In [59]:
flights_small2 = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features').transform(flights_small2)
flights_small2.show(2, truncate=False)

+------+-------+-------------+--------+----------------------+
|km    |org_idx|org_dummy    |duration|features              |
+------+-------+-------------+--------+----------------------+
|253.0 |0.0    |(7,[0],[1.0])|51      |(8,[0,1],[253.0,1.0]) |
|1188.0|0.0    |(7,[0],[1.0])|127     |(8,[0,1],[1188.0,1.0])|
+------+-------+-------------+--------+----------------------+
only showing top 2 rows



In [60]:
flights_train_small2, flights_test_small2 = flights_small2.randomSplit([0.8, 0.2], seed=17)

In [61]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train_small2)

# Create predictions for the testing data
predictions = regression.transform(flights_test_small2)

# Calculate the RMSE on testing data
RegressionEvaluator(labelCol='duration').evaluate(predictions)

11.032683206654783

## Interpreting coefficients

Remember that origin airport, org, has eight possible values (ORD, SFO, JFK, LGA, SMF, SJC, TUS and OGG) which have been one-hot encoded to seven dummy variables in org_dummy.

The values for km and org_dummy have been assembled into features, which has eight columns with sparse representation. Column indices in features are as follows:

- 0 — km
- 1 — ORD
- 2 — SFO
- 3 — JFK
- 4 — LGA
- 5 — SMF
- 6 — SJC and
- 7 — TUS.

Note that OGG does not appear in this list because it is the reference level for the origin airport category.

In this exercise you'll be using the intercept and coefficients attributes to interpret the model.

The coefficients attribute is a list, where the first element indicates how flight duration changes with flight distance.

- Find the average speed in km per hour. This will be different to the value that you got earlier because your model is now more sophisticated.
- What's the average time on the ground at OGG?
- What's the average time on the ground at JFK?
- What's the average time on the ground at LGA?

In [63]:
regression.coefficients

DenseVector([0.0743, 28.3871, 20.3442, 52.6047, 46.7801, 15.7718, 18.1336, 18.0337])

In [64]:
# Average speed in km per hour
# The first coefficient, regression.coefficients[0], is in minutes per km. 
# Invert this to get speed in km per minute then multiply by 60 to get km per hour.
avg_speed_hour = 60 / regression.coefficients[0]
print(avg_speed_hour)

# Average minutes on ground at OGG
# OGG is the reference level for org_dummy, so this delay is simply regression.intercept.
inter = regression.intercept
print(inter)

# Average minutes on ground at JFK
# You need to add the coefficient for JFK to regression.intercept.
avg_ground_jfk = inter + regression.coefficients[3]
print(avg_ground_jfk)

# Average minutes on ground at LGA
# You need to add the coefficient for LGA to regression.intercept.
avg_ground_lga = inter + regression.coefficients[4]
print(avg_ground_lga)

807.8981465028214
15.927742633535276
68.53240648000819
62.70783982398586


You're going to spend over an hour on the ground at JFK or LGA but only around 15 minutes at OGG.

## Bucketing & Engineering

<img src="images/spark5_074.png" alt="" style="width: 800px;"/>

<img src="images/spark5_075.png" alt="" style="width: 800px;"/>

<img src="images/spark5_076.png" alt="" style="width: 800px;"/>

<img src="images/spark5_077.png" alt="" style="width: 800px;"/>

<img src="images/spark5_078.png" alt="" style="width: 800px;"/>

<img src="images/spark5_079.png" alt="" style="width: 800px;"/>

<img src="images/spark5_080.png" alt="" style="width: 800px;"/>

<img src="images/spark5_081.png" alt="" style="width: 800px;"/>

<img src="images/spark5_082.png" alt="" style="width: 800px;"/>

<img src="images/spark5_083.png" alt="" style="width: 800px;"/>

<img src="images/spark5_084.png" alt="" style="width: 800px;"/>

## Bucketing departure time

Time of day data are a challenge with regression models. They are also a great candidate for bucketing.

In this lesson you will convert the flight departure times from numeric values between 0 (corresponding to 00:00) and 24 (corresponding to 24:00) to binned values. You'll then take those binned values and one-hot encode them.

- Create a bucketizer object with bin boundaries which correspond to 0:00, 03:00, 06:00, ..., 24:00. Specify input column as depart and output column as depart_bucket.
- Bucket the departure times. Show the first five values for depart and depart_bucket.
- Create a one-hot encoder object. Specify output column as depart_dummy.
- Train the encoder on the data and then use it to convert the bucketed departure times to dummy variables. Show the first five values for depart, depart_bucket and depart_dummy.

```
flights.show(2)
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 2 rows
```

In [65]:
flights.show(2)

+---+---+---+-------+---+----+------+--------+-----+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|
+---+---+---+-------+---+----+------+--------+-----+
| 10| 10|  1|     OO|ORD| 157|  8.18|      51|   27|
| 11| 22|  1|     OO|ORD| 738|  7.17|     127|  -19|
+---+---+---+-------+---+----+------+--------+-----+
only showing top 2 rows



In [66]:
from pyspark.ml.feature import Bucketizer, OneHotEncoderEstimator

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=[0, 3, 6, 9, 12, 15, 18, 21, 24], inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
bucketed = buckets.transform(flights)
bucketed.select('depart', 'depart_bucket').show(5)

# Create a one-hot encoder
onehot = OneHotEncoderEstimator(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed departure times
flights_onehot = onehot.fit(bucketed).transform(bucketed)
flights_onehot.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+------+-------------+
|depart|depart_bucket|
+------+-------------+
|  8.18|          2.0|
|  7.17|          2.0|
| 21.17|          7.0|
| 12.92|          4.0|
| 13.33|          4.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
|  8.18|          2.0|(7,[2],[1.0])|
|  7.17|          2.0|(7,[2],[1.0])|
| 21.17|          7.0|    (7,[],[])|
| 12.92|          4.0|(7,[4],[1.0])|
| 13.33|          4.0|(7,[4],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



## Flight duration model: Adding departure time

In the previous exercise the departure time was bucketed and converted to dummy variables. Now you're going to include those dummy variables in a regression model for flight duration.

The data are in flights. The km, org_dummy and depart_dummy columns have been assembled into features, where km is index 0, org_dummy runs from index 1 to 7 and depart_dummy from index 8 to 14.

The data have been split into training and testing sets and a linear regression model, regression, has been built on the training data. Predictions have been made on the testing data and are available as predictions.

- Find the RMSE for predictions on the testing data.
- Find the average time spent on the ground for flights departing from OGG between 21:00 and 24:00.
- Find the average time spent on the ground for flights departing from OGG between 00:00 and 03:00.
- Find the average time spent on the ground for flights departing from JFK between 00:00 and 03:00.

```
flights.show(2)
+---+---+---+-------+------+---+------+--------+-----+------+-------+-------------+-------------+-------------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|org_idx|    org_dummy|depart_bucket| depart_dummy|            features|
+---+---+---+-------+------+---+------+--------+-----+------+-------+-------------+-------------+-------------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|3465.0|    2.0|(7,[2],[1.0])|          3.0|(7,[3],[1.0])|(15,[0,3,11],[346...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30| 509.0|    0.0|(7,[0],[1.0])|          5.0|(7,[5],[1.0])|(15,[0,1,13],[509...|
+---+---+---+-------+------+---+------+--------+-----+------+-------+-------------+-------------+-------------+--------------------+
only showing top 2 rows
```

```
Feature columns:

 0 — km
 1 — ORD
 2 — SFO
 3 — JFK
 4 — LGA
 5 — SJC
 6 — SMF
 7 — TUS
 8 — 00:00 to 03:00
 9 — 03:00 to 06:00
10 — 06:00 to 09:00
11 — 09:00 to 12:00
12 — 12:00 to 15:00
13 — 15:00 to 18:00
14 — 18:00 to 21:00
```

In [69]:
flights_indexed.show(2)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|        2.0|    0.0|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|        2.0|    0.0|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
only showing top 2 rows



In [82]:
buckets = Bucketizer(splits=[0, 3, 6, 9, 12, 15, 18, 21, 24], inputCol='depart', outputCol='depart_bucket')
bucketed = buckets.transform(flights_indexed)
onehot = OneHotEncoderEstimator(inputCols=['depart_bucket'], outputCols=['depart_dummy'])
flights2 = onehot.fit(bucketed).transform(bucketed)
onehot2 = OneHotEncoderEstimator(inputCols=['org_idx'], outputCols=['org_dummy'])
flights2 = onehot2.fit(flights2).transform(flights2)
flights2.show(2)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|depart_bucket| depart_dummy|    org_dummy|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|        2.0|    0.0|          2.0|(7,[2],[1.0])|(7,[0],[1.0])|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|        2.0|    0.0|          2.0|(7,[2],[1.0])|(7,[0],[1.0])|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+
only showing top 2 rows



In [83]:
flights2 = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy'], outputCol='features').transform(flights2)
flights2_train, flights2_test = flights2.randomSplit([0.8, 0.2], seed=17)

In [84]:
regression = LinearRegression(labelCol='duration').fit(flights2_train)
predictions = regression.transform(flights2_test)

In [85]:
# Find the RMSE on testing data
from pyspark.ml.evaluation import RegressionEvaluator
RegressionEvaluator(labelCol='duration').evaluate(predictions)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights departing between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[8] + regression.coefficients[3]
print(avg_night_jfk)

10.00470660357773
-4.374897246310109
47.64814727637918


Adding departure time resulted in a smaller RMSE. Nice!

## Regularization

<img src="images/spark5_085.png" alt="" style="width: 800px;"/>

<img src="images/spark5_086.png" alt="" style="width: 800px;"/>

<img src="images/spark5_087.png" alt="" style="width: 800px;"/>

<img src="images/spark5_088.png" alt="" style="width: 800px;"/>

<img src="images/spark5_089.png" alt="" style="width: 800px;"/>

<img src="images/spark5_090.png" alt="" style="width: 800px;"/>

<img src="images/spark5_091.png" alt="" style="width: 800px;"/>

<img src="images/spark5_092.png" alt="" style="width: 800px;"/>

## Flight duration model: More features!

Let's add more features to our model. This will not necessarily result in a better model. Adding some features might improve the model. Adding other features might make it worse.

`More features will always make the model more complicated and difficult to interpret`.

These are the features you'll include in the next model:

- km
- org (origin airport, one-hot encoded, 8 levels)
- depart (departure time, binned in 3 hour intervals, one-hot encoded, 8 levels)
- dow (departure day of week, one-hot encoded, 7 levels) and
- mon (departure month, one-hot encoded, 12 levels).

These have been assembled into the features column, which is a sparse representation of 32 columns (remember one-hot encoding produces a number of columns which is one fewer than the number of levels).

The data are available as flights, randomly split into flights_train and flights_test. The object predictions is also available.

- Fit a linear regression model to the training data.
- Generate predictions for the testing data.
- Calculate the RMSE on the testing data.
- Look at the model coefficients. Are any of them zero?

```
Subset from the flights DataFrame:

+--------------------------------------------+--------+
|features                                    |duration|
+--------------------------------------------+--------+
|(32,[0,3,11],[3465.0,1.0,1.0])              |351     |
|(32,[0,1,13,17,21],[509.0,1.0,1.0,1.0,1.0]) |82      |
|(32,[0,2,10,19,23],[542.0,1.0,1.0,1.0,1.0]) |82      |
|(32,[0,1,11,16,30],[1989.0,1.0,1.0,1.0,1.0])|195     |
|(32,[0,1,10,20,25],[415.0,1.0,1.0,1.0,1.0]) |65      |
+--------------------------------------------+--------+
only showing top 5 rows
```

```
flights.show(2)
+---+---+---+-------+------+---+------+--------+-----+------+-------+-------------+-------------+-------------+-------------+--------------+--------------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|    km|org_idx|    org_dummy|depart_bucket| depart_dummy|    dow_dummy|     mon_dummy|            features|
+---+---+---+-------+------+---+------+--------+-----+------+-------+-------------+-------------+-------------+-------------+--------------+--------------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|3465.0|    2.0|(7,[2],[1.0])|          3.0|(7,[3],[1.0])|    (6,[],[])|    (11,[],[])|(32,[0,3,11],[346...|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30| 509.0|    0.0|(7,[0],[1.0])|          5.0|(7,[5],[1.0])|(6,[2],[1.0])|(11,[0],[1.0])|(32,[0,1,13,17,21...|
+---+---+---+-------+------+---+------+--------+-----+------+-------+-------------+-------------+-------------+-------------+--------------+--------------------+
only showing top 2 rows
```

In [86]:
# locally, I have only 15 features instead of 32
flights2.show(2)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+--------------------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|depart_bucket| depart_dummy|    org_dummy|            features|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+--------------------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|        2.0|    0.0|          2.0|(7,[2],[1.0])|(7,[0],[1.0])|(15,[0,1,10],[253...|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|        2.0|    0.0|          2.0|(7,[2],[1.0])|(7,[0],[1.0])|(15,[0,1,10],[118...|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+--------------------+
only showing top 2 rows



In [87]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Fit linear regression model to training data
regression = LinearRegression(labelCol='duration').fit(flights2_train)

# Make predictions on testing data
predictions = regression.transform(flights2_test)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

The test RMSE is 10.658023625236448
[0.07431564263161233,27.411972653788936,20.281085851147388,52.02304452268929,46.09559165221516,15.354895160481403,17.576351930860596,17.658914170830375,-14.37960384988784,0.526955806213976,4.2707310341421945,7.316813537461249,4.915097563096843,9.11586198206896,9.118244091316992]


With all those non-zero coefficients the model is a little hard to interpret!

## Flight duration model: Regularisation!

In the previous exercise you added more predictors to the flight duration model. The model performed well on testing data, but with so many coefficients it was difficult to interpret.

In this exercise you'll use `Lasso regression` (regularized with a `L1 penalty`) to create a more parsimonious model. Many of the coefficients in the resulting model will be set to zero. This means that only a subset of the predictors actually contribute to the model. Despite the simpler model, it still produces a good RMSE on the testing data.

You'll use a specific value for the regularization strength. Later you'll learn how to find the best value using cross validation.

The data (same as previous exercise) are available as flights2, randomly split into flights2_train and flights2_test.

- Fit a linear regression model to the training data.
- Calculate the RMSE on the testing data.
- Look at the model coefficients.
- Get the count of coefficients equal to 0.

In [88]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Fit Lasso model (α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights2_train)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(regression.transform(flights2_test))
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

# Number of zero coefficients
zero_coeff = sum([beta == 0 for beta in regression.coefficients])
print("Number of ceofficients equal to 0:", zero_coeff)

The test RMSE is 11.517020719626478
[0.07350295986840152,5.641160609271034,0.0,29.23319932748944,22.23833621413661,-2.1006774075213097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0456121619917145,1.243075237505768]
Number of ceofficients equal to 0: 8


Regularisation produced a far simpler model with similar test performance.

---
<a id='ens'></a>

## Ensembles & Pipelines

<img src="images/spark5_093.png" alt="" style="width: 800px;"/>

<img src="images/spark5_094.png" alt="" style="width: 800px;"/>

<img src="images/spark5_095.png" alt="" style="width: 800px;"/>

<img src="images/spark5_096.png" alt="" style="width: 800px;"/>

<img src="images/spark5_097.png" alt="" style="width: 800px;"/>

<img src="images/spark5_098.png" alt="" style="width: 800px;"/>

<img src="images/spark5_099.png" alt="" style="width: 800px;"/>

<img src="images/spark5_100.png" alt="" style="width: 800px;"/>

## Flight duration model: Pipeline stages

You're going to create the stages for the flights duration model pipeline. You will use these in the next exercise to build a pipeline and to create a regression model.

- Create an indexer to convert the 'org' column into an indexed column called 'org_idx'.
- Create a one-hot encoder to convert the 'org_idx' and 'dow' columns into dummy variable columns called 'org_dummy' and 'dow_dummy'.
- Create an assembler which will combine the 'km' column with the two dummy variable columns. The output column should be called 'features'.
- Create a linear regression object to predict flight duration.

```
The first few rows of the flights DataFrame:

+---+---+---+-------+------+---+------+--------+-----+------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|km    |
+---+---+---+-------+------+---+------+--------+-----+------+
|11 |20 |6  |US     |19    |JFK|9.48  |351     |null |3465.0|
|0  |22 |2  |UA     |1107  |ORD|16.33 |82      |30   |509.0 |
|2  |20 |4  |UA     |226   |SFO|6.17  |82      |-8   |542.0 |
|9  |13 |1  |AA     |419   |ORD|10.33 |195     |-5   |1989.0|
|4  |2  |5  |AA     |325   |ORD|8.92  |65      |null |415.0 |
+---+---+---+-------+------+---+------+--------+-----+------+
only showing top 5 rows
```

In [89]:
flights.show(5)

+---+---+---+-------+---+----+------+--------+-----+
|mon|dom|dow|carrier|org|mile|depart|duration|delay|
+---+---+---+-------+---+----+------+--------+-----+
| 10| 10|  1|     OO|ORD| 157|  8.18|      51|   27|
| 11| 22|  1|     OO|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|SJC| 386| 12.92|      85|   22|
|  3| 28|  1|     B6|LGA|1076| 13.33|     182|   70|
+---+---+---+-------+---+----+------+--------+-----+
only showing top 5 rows



In [90]:
flights_km = flights.withColumn('km', round(flights.mile * 1.60934, 0)) \
                    .drop('mile')
flights_km.show(5)

+---+---+---+-------+---+------+--------+-----+------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|
+---+---+---+-------+---+------+--------+-----+------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|
|  2| 14|  5|     B6|JFK| 21.17|     365|   60|3618.0|
|  5| 25|  3|     WN|SJC| 12.92|      85|   22| 621.0|
|  3| 28|  1|     B6|LGA| 13.33|     182|   70|1732.0|
+---+---+---+-------+---+------+--------+-----+------+
only showing top 5 rows



In [91]:
# Convert categorical strings to index values
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# One-hot encode index values
onehot = OneHotEncoderEstimator(
    inputCols=['org_idx', 'dow'],
    outputCols=['org_dummy', 'dow_dummy']
)

# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'], outputCol='features')

# A linear regression object
regression = LinearRegression(labelCol='duration')

## Flight duration model: Pipeline model

You're now ready to put those stages together in a pipeline.

You'll construct the pipeline and then train the pipeline on the training data. This will apply each of the individual stages in the pipeline to the training data in turn. `None of the stages will be exposed to the testing data at all: there will be no leakage!`

Once the entire pipeline has been trained it will then be used to make predictions on the testing data.

The data are available as flights, which has been randomly split into flights_train and flights_test.

In [92]:
flights_train, flights_test = flights_km.randomSplit([0.8, 0.2], seed=17)

In [ ]:
# Terminate the cluster
spark.stop()

In [ ]:
<img src="images/spark5_101.png" alt="" style="width: 800px;"/>

In [ ]:
---
<a id='intro'></a>